# UTS
Lakukan analisa klasifikasi terhadap data abstrak prodi teknik informatika pta.trunojoyo.ac.id dengan menggunakan model klasifikasi naïve bayes dan KNN denga preprossing reduksi dimensi PCA (principal component analysis)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
import pandas as pd
df=pd.read_excel('/content/drive/MyDrive/prosaindata/tugas/PTA.xlsx')
display(df)
df.isnull().sum()


,NPM,Judul,Abstrak,Prodi,Label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,komputasi
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi
...,...,...,...,...,...
806,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...,Teknik Informatika,komputasi
807,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...,Teknik Informatika,PBA
808,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...,Teknik Informatika,komputasi
809,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...,Teknik Informatika,komputasi


NPM        0
Judul      0
Abstrak    0
Prodi      0
Label      0
dtype: int64

In [51]:
#mengecek total label
df['Label'].value_counts()

komputasi    574
RPL          198
PBA           39
Name: Label, dtype: int64

In [52]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB

# mengubah kolom "Abstrak" dari dataframe df menjadi list yang berisi token yang telah dipisahkan. 
# Pengubahan dilakukan dengan mengaplikasikan fungsi split() pada setiap teks pada kolom "Abstrak". 
# lambda x digunakan untuk mengakses setiap teks pada kolom "Abstrak" secara individual.
df['Abstrak'] = df['Abstrak'].apply(lambda x: x.split())
print(df['Abstrak'].head())

# membagi dataset ke dalam train set dan test set. Data yang akan digunakan untuk training disimpan dalam variabel X_train dan y_train
# sementara data yang akan digunakan untuk pengujian atau testing disimpan dalam variabel X_test dan y_test. 
# Parameter test_size menunjukkan ukuran dari test set yang akan dihasilkan.
X_train, X_test, y_train, y_test = train_test_split(df['Abstrak'], df['Label'], test_size=0.2, random_state=42)

# inisialisasi model Word2Vec. X_train adalah list token yang dihasilkan dari bagian sebelumnya
# vector_size menunjukkan dimensi yang ingin dihasilkan pada model
# min_count adalah jumlah minimum kata yang ingin ditampung dalam model.
w2v_model = Word2Vec(X_train, vector_size=100, min_count=1)

# Define Word2Vec transformer
class Word2VecTransformer:
    def __init__(self, w2v_model):
        self.w2v_model = w2v_model
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return np.array([np.mean([self.w2v_model.wv[w] for w in words if w in self.w2v_model.wv] or [np.zeros(self.w2v_model.vector_size)], axis=0) for words in X])
        
# Define preprocessing pipeline
pipeline = Pipeline([
    ('word2vec', Word2VecTransformer(w2v_model)),
    ('pca', PCA(n_components=50)),
    ('scaler', StandardScaler()),
    ('classifier', None)
])

# Transform data using the pipeline
X_pca = pipeline.fit_transform(X_train)

# Split data into train and test sets for PCA data
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y_train, test_size=0.2, random_state=42)

# Train KNN model
# model KNN dibuat dengan n_neighbors=5, artinya model akan mencari 5 tetangga terdekat dari setiap data baru. 
# model dilatih menggunakan data latih hasil transformer dengan knn.fit(X_train_pca, y_train_pca).
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train_pca)

# Train Gaussian Naive Bayes model
nb = GaussianNB()
nb.fit(X_train_pca, y_train_pca)

# Evaluate models
y_pred_knn = knn.predict(X_test_pca)
acc_knn = accuracy_score(y_test_pca, y_pred_knn)
print("Accuracy KNN:", acc_knn)

y_pred_nb = nb.predict(X_test_pca)
acc_nb = accuracy_score(y_test_pca, y_pred_nb)
print("Accuracy Gaussian Naive Bayes:", acc_nb)


0    [Sistem, informasi, akademik, (SIAKAD), merupa...
1    [Berjalannya, koneksi, jaringan, komputer, den...
2    [Web, server, adalah, sebuah, perangkat, lunak...
3    [Penjadwalan, kuliah, di, Perguruan, Tinggi, m...
4    [Seiring, perkembangan, teknologi, yang, ada, ...
Name: Abstrak, dtype: object
Accuracy KNN: 0.7846153846153846
Accuracy Gaussian Naive Bayes: 0.7769230769230769
